# Auto Encoder for LCH Olympics

In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image
import torch.nn.functional as F
from sklearn import metrics
import numpy as np
import colorsys
import matplotlib
from matplotlib.lines import Line2D
from matplotlib.font_manager import FontProperties
import pandas as pd
#from pyjet import cluster,DTYPE_PTEPM
import random


/tmpscratch/ktamenar/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device
device = get_device()
device

'cuda:0'

In [3]:
torch.cuda.set_device(device)
torch.cuda.empty_cache()

In [4]:
#import os
#os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:<512>"
#torch.backends.cuda.max_memory_split = 256

In [5]:
#events= pd.read_hdf('events_LHCO2020_BlackBox1.h5')
events= pd.read_hdf('events_1M.h5')

In [6]:
#events = events.T
np.shape(events)
#events.head(10)

(1000000, 477)

In [7]:
#Separates de events into test, validation and training

events_tra, events_val, events_tes = np.split(events, [int(.6*len(events)), int(.8*len(events))])

print(len(events_tra))
print(len(events_val))
print(len(events_tes))

600000
200000
200000


In [8]:
#Autoencoder example 

class Autoencoder1(nn.Module):
    def __init__(self):
        super(Autoencoder1, self).__init__()
        # encoder
        
        n1,n2,n3,n4,n5,n6 = 477, 400, 300, 200, 100, 50
        
        self.enc1 = nn.Linear(in_features= n1, out_features=n2)
        self.enc2 = nn.Linear(in_features=n2, out_features=n3)
        self.enc3 = nn.Linear(in_features=n3, out_features=n4)
        self.enc4 = nn.Linear(in_features=n4, out_features=n5)
        self.enc5 = nn.Linear(in_features=n5, out_features=n6)
        # decoder 
        self.dec1 = nn.Linear(in_features=n6, out_features=n5)
        self.dec2 = nn.Linear(in_features=n5, out_features=n4)
        self.dec3 = nn.Linear(in_features=n4, out_features=n3)
        self.dec4 = nn.Linear(in_features=n3, out_features=n2)
        self.dec5 = nn.Linear(in_features=n2, out_features= n1)
    def forward(self, x):
        x = F.relu(self.enc1(x))
        x = F.relu(self.enc2(x))
        x = F.relu(self.enc3(x))
        x = F.relu(self.enc4(x))
        x = F.relu(self.enc5(x))
        x = F.relu(self.dec1(x))
        x = F.relu(self.dec2(x))
        x = F.relu(self.dec3(x))
        x = F.relu(self.dec4(x))
        x = F.relu(self.dec5(x))
        return x
model = Autoencoder1()
#print(model)

In [9]:
model = model.cuda()

optimizer = torch.optim.Adam(model.parameters(),
        lr = 1e-3,
        weight_decay = 1e-8)

#optimizer = optim.Adam(model.parameters(), lr=0.001)

criterion = nn.MSELoss()

epochs = 100
batch_size = 512

#Events for training set

train_losses = []

trainset = torch.tensor(events_tra.values)
trainset=trainset.to(torch.float32)
#print(trainset.shape)


trainloader = DataLoader(
    dataset=trainset, 
    batch_size=batch_size,
    shuffle=True,
)
print(trainloader)
print(len(trainloader))

print("******************************")

#Events for validation set

val_losses = []

valset = torch.tensor(events_val.values)
valset=valset.to(torch.float32)

valloader = DataLoader(
    dataset = valset, 
    batch_size = batch_size,
    shuffle = False,
)

print(valloader)
print(len(valloader))
print("********************************")
type(trainloader.dataset)
type(valloader.dataset)

1172
******************************
391
********************************


torch.Tensor

In [10]:
yv=[]
xv=[]
xt=[]
yt=[] 

def train(epoch):
    model.train()
    train_loss = 0
    val_loss=0
    
    for data in trainloader:
        data = data.cuda()
        xt.append(data)         #input
        optimizer.zero_grad()
        y=model(data)  #output
        yt.append(y)
        loss = criterion(y, data) #comparision between input and target data(output)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    #if epoch % 2 == 0:        
    print('====> Epoch: {} Average_training_loss: {:.4f}'.format(
    epoch, train_loss / len(trainloader.dataset)))
    train_losses.append(train_loss / len(trainloader.dataset))
    
    
    for data in valloader:
        data = data.cuda()
        xv.append(data)
        optimizer.zero_grad()
        y=model(data)
        yv.append(y)
        loss = criterion(y, data)
        loss.backward()
        val_loss += loss.item()
        optimizer.step()
        #avg_vloss = running_vloss / (i + 1)
        #print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))
    print('====> Epoch: {} Average_validaition_loss: {:.4f}'.format(
    epoch, val_loss / len(valloader.dataset)))
    val_losses.append(val_loss / len(valloader.dataset))
    print('*********************************************************')


In [11]:
for epoch in range(1, epochs + 1):
    train(epoch)

====> Epoch: 1 Average_training_loss: 0.9983
====> Epoch: 1 Average_validaition_loss: 0.2457
*********************************************************
====> Epoch: 2 Average_training_loss: 0.1958
====> Epoch: 2 Average_validaition_loss: 0.1821
*********************************************************
====> Epoch: 3 Average_training_loss: 0.1769
====> Epoch: 3 Average_validaition_loss: 0.1713
*********************************************************
====> Epoch: 4 Average_training_loss: 0.1656
====> Epoch: 4 Average_validaition_loss: 0.1543
*********************************************************
====> Epoch: 5 Average_training_loss: 0.1506
====> Epoch: 5 Average_validaition_loss: 0.1487
*********************************************************
====> Epoch: 6 Average_training_loss: 0.1467
====> Epoch: 6 Average_validaition_loss: 0.1438
*********************************************************
====> Epoch: 7 Average_training_loss: 0.1402


RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 22.16 GiB total capacity; 19.42 GiB already allocated; 1.88 MiB free; 20.85 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:

plt.figure()
plt.plot(train_losses, alpha=1.0, linewidth=1.5, label="training_loss")
plt.plot(val_losses, alpha=1.0, linewidth=1.5, label="validation_loss")
plt.title('Autoencoder Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.legend(loc='upper right')
plt.savefig('Loss.png')

#fig.tight_layout()
#fig.savefig( "Loss.png", bbox_inches='tight' )

# Visualizing imput vs output

In [ ]:
def plot_heatmap(input, output, label):

    input = input.reshape(159,3)
    output = output.reshape(159,3)

    input = pd.DataFrame(input, columns = ["pT","eta","phi"])
    input = input.loc[~(input==0).all(axis=1)]
    output = pd.DataFrame(output, columns = ["pT","eta","phi"])
    output = output.loc[~(output==0).all(axis=1)]

    fig = plt.figure(figsize=(10,10))
    ax = fig.gca(projection='3d')
    surf = ax.bar3d(input['eta'],input['phi'],5,0.2,0.2,input['pT'],color='g',shade=True)
    in_proxy = plt.Rectangle((0, 0), 1, 1, fc="g")
    ax.set_xlim(-4.5,4.5)
    ax.set_ylim(-3.2,3.2)
    ax.set_xlabel("eta")
    ax.set_ylabel("phi")
    ax.set_zlabel("pT")
    surf2 = ax.bar3d(output['eta'],output['phi'],5,0.2,0.2,output['pT'],shade=True)
    out_proxy = plt.Rectangle((0, 0), 1, 1, fc="b")
    ax.legend([in_proxy,out_proxy],['input event','AE output event'])
    plt.show()
    plt.close()

In [ ]:
def plot_heatmap_blue(input, output, label):

    input = input.reshape(159,3)
    output = output.reshape(159,3)

    input = pd.DataFrame(input, columns = ["pT","eta","phi"])
    input = input.loc[~(input==0).all(axis=1)]
    output = pd.DataFrame(output, columns = ["pT","eta","phi"])
    output = output.loc[~(output==0).all(axis=1)]

    fig = plt.figure(figsize=(10,10))
    ax = fig.gca(projection='3d')
    #surf = ax.bar3d(input['eta'],input['phi'],5,0.2,0.2,input['pT'],color='g',shade=True)
    in_proxy = plt.Rectangle((0, 0), 1, 1, fc="g")
    ax.set_xlim(-4.5,4.5)
    ax.set_ylim(-3.2,3.2)
    ax.set_xlabel("eta")
    ax.set_ylabel("phi")
    ax.set_zlabel("pT")
    surf2 = ax.bar3d(output['eta'],output['phi'],5,0.2,0.2,output['pT'],shade=True)
    out_proxy = plt.Rectangle((0, 0), 1, 1, fc="b")
    ax.legend([in_proxy,out_proxy],['input event','AE output event'])
    plt.show()
    plt.close()

In [ ]:
def plot_heatmap_green(input, output, label):

    input = input.reshape(159,3)
    output = output.reshape(159,3)

    input = pd.DataFrame(input, columns = ["pT","eta","phi"])
    input = input.loc[~(input==0).all(axis=1)]
    output = pd.DataFrame(output, columns = ["pT","eta","phi"])
    output = output.loc[~(output==0).all(axis=1)]

    fig = plt.figure(figsize=(10,10))
    ax = fig.gca(projection='3d')
    surf = ax.bar3d(input['eta'],input['phi'],5,0.2,0.2,input['pT'],color='g',shade=True)
    in_proxy = plt.Rectangle((0, 0), 1, 1, fc="g")
    ax.set_xlim(-4.5,4.5)
    ax.set_ylim(-3.2,3.2)
    ax.set_xlabel("eta")
    ax.set_ylabel("phi")
    ax.set_zlabel("pT")
    #surf2 = ax.bar3d(output['eta'],output['phi'],5,0.2,0.2,output['pT'],shade=True)
    out_proxy = plt.Rectangle((0, 0), 1, 1, fc="b")
    ax.legend([in_proxy,out_proxy],['input event','AE output event'])
    plt.show()
    plt.close()

In [ ]:
n=15


yn = yt[n][:,:]
print(yn.shape)
yn = yn.detach().numpy()
xn = xt[n][:,:].detach().numpy()
print(xn.shape)

In [ ]:
plot_heatmap(xn[0],yn[0],"all")

In [ ]:
plot_heatmap_blue(xn[0],yn[0],"blue")

In [ ]:
plot_heatmap_green(xn[0],yn[0],"green")